<a href="https://colab.research.google.com/github/alvarofpinheiro/k_nearest_neighbor/blob/main/Simulacao1_Cenario2_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

K-Nearest Neighbors  (KNN): método K vizinhos mais próximos foi proposto por Fukunaga e Narendra em 1975, se tornando um dos algoritmos para solução de problemas da classe de classificação mais usados, pois possui uma abordagem simples de ser implementada, apesar que para grandes volumes de dados irá usar muito recursos de hardware devido a base de sua lógica que consiste no cálculo de distância, e essa é uma tarefa custosa para o hardware, consumindo muito tempo computacional, e é um método que depende no fator K, assim quanto maior, mais recurso de hardware será consumido.

Fukunaga, K.; Narendra, P. A Branch and Bound Algorithm for Computing k-Nearest Neighbors.  IEEE Transactions on Computers Volume: C-24, Issue: 7. https://doi.org/10.1109/T-C.1975.224297, 1975.

Explicação do algoritmo KNN: https://youtu.be/oafK8OGu_Vk

In [ ]:
#instalar biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importar biblioteca Orange Canvas
import Orange

In [ ]:
#importar dados do disco local
from google.colab import files  
files.upload()

In [ ]:
#instanciar objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/simulacao1.csv")

In [ ]:
#exibir os primeiros 5 registros para uma rápida análise da importação dos dados
for d in dados[:5]:
  print(d)

In [ ]:
#explorar os metadados da base de dados importada
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)

In [ ]:
#explorar domínios dos atributos (campos da base de dados)
dados.domain.attributes

In [ ]:
#explorar as classes da base de dados
dados.domain.class_var

In [ ]:
#explorar dados (quantidade de campos e registros da base de dados)
print("Campos:", ", ".join(c.name for c in dados.domain.attributes))
qtde100 = len(dados)
qtde70 = len(dados) * 70 / 100
qtde30 = len(dados) * 30 / 100
print("Qtde 100%:", qtde100)
print("Qtde  70%:", qtde70)
print("Qtde  30%:", qtde30)
amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])
print("Registros:", len(dados))
print("Registros:", len(amostra))

**Hiperparâmetros do K-Nearest Neighbors**

n_neighbors: valor correspondente a quantidade de vizinhos mais próximos.

metric: métrica a ser utilizada para cálculo da distância com o default sendo a euclidiana.

weights: determina se o peso será distance onde os vizinhos mais próximos têm influência maior, ou se uniform onde todos os pontos em cada vizinhança são ponderados igualmente.

preprocessors: é usado o pré-processamento padrão quando nenhum outro pré-processador é fornecido. Ele os executa na seguinte ordem: remove instâncias com valores de destino desconhecidos; continua variáveis categóricas (com codificação one-hot); remove colunas vazias; imputa valores ausentes com valores médios.

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica KNN Cenário1
knn = Orange.classification.KNNLearner(
    n_neighbors=5, 
    metric='euclidean', 
    weights='distance', 
    algorithm='auto', 
    metric_params=None, 
    preprocessors=None)

In [ ]:
#ligar técnica KNN com os dados da simulação
dados_knn = knn(dados)

In [ ]:
#treinar e testar técnica KNN com os dados com 5 partes (folders)
avalia_knn = Orange.evaluation.CrossValidation(dados, [knn], k=5)

In [ ]:
#avaliar os indicadores para o KNN com base nas métricas de classificação
print("Acurácia:  %.3f" % Orange.evaluation.scoring.CA(avalia_knn)[0])
print("Precisão:  %.3f" % Orange.evaluation.scoring.Precision(avalia_knn)[0])
print("Revocação: %.3f" % Orange.evaluation.scoring.Recall(avalia_knn)[0])
print("F1:        %.3f" % Orange.evaluation.scoring.F1(avalia_knn)[0])
print("ROC:       %.3f" % Orange.evaluation.scoring.AUC(avalia_knn)[0])

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Support Vector Machine (SVM) Cenário2
svm = Orange.classification.SVMLearner(
    C=1.0, 
    kernel='rbf', 
    degree=3, 
    gamma='auto', 
    coef0=0.0, 
    shrinking=True, 
    probability=False, 
    tol=0.001, 
    cache_size=2000, 
    max_iter=-1, 
    preprocessors=None)

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Random Forest (RF) Cenário3
rf = Orange.classification.RandomForestLearner(
    n_estimators=10, 
    criterion='gini', 
    max_depth=None, 
    min_samples_split=2, 
    min_samples_leaf=1, 
    min_weight_fraction_leaf=0.0, 
    max_features='auto', 
    max_leaf_nodes=None, 
    bootstrap=True, 
    oob_score=False, 
    n_jobs=1, 
    random_state=None, 
    verbose=0, 
    class_weight=None, 
    preprocessors=None)

In [ ]:
#validar o aprendizado para as 3 técnicas
aprendizado = [knn, svm, rf]
avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)

In [ ]:
#imprimir os indicadores para as 3 técnicas
print(" " * 10 + " | ".join("%-4s" % learner.name for learner in aprendizado))
print("Acurácia  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.CA(avaliacao)))
print("Precisão  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Precision(avaliacao)))
print("Revocação %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Recall(avaliacao)))
print("F1        %s" % " | ".join("%.2f" % s for s in Orange.evaluation.F1(avaliacao)))
print("ROC       %s" % " | ".join("%.2f" % s for s in Orange.evaluation.AUC(avaliacao)))